In [1]:
import qutip as qtp
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import matplotlib.pyplot as plt
from IPython.display import display, Math

## Setup parameters

In [2]:
num_mode_bits = 2
mode_dim = 2 ** num_mode_bits
num_modes = mode_dim - 1 # 7 site lattice
pmin = -mode_dim // 2 + 1
pmax = mode_dim // 2 - 1
null_mode = 2 ** (num_mode_bits - 1)
max_num_particles = 3

## Operators

In [3]:
def particle_ket(p):
    p_label = (p + mode_dim) % mode_dim
    return qtp.basis(mode_dim, p_label)

null_ket = qtp.basis(mode_dim, null_mode)
null_proj = null_ket.proj()
nonnull_proj = qtp.qeye(mode_dim) - null_proj

def particle_creation(ipart, p):
    cr = particle_ket(p) * null_ket.dag()
    p0 = null_proj
    
    ops = [nonnull_proj for _ in range(ipart)]
    ops.append(cr)
    ops += [p0 for _ in range(ipart + 1, max_num_particles)]
        
    return qtp.tensor(ops[::-1])

def swap(source, target):
    ops = [nonnull_proj for _ in range(target)]
    
    if source == target:
        ops.append(nonnull_proj)
    else:
        swap_block = qtp.Qobj()
        for k in range(mode_dim):
            if k == null_mode:
                continue
            
            for l in range(mode_dim):
                if l == null_mode:
                    continue

                swap_ops = [qtp.basis(mode_dim, k) * qtp.basis(mode_dim, l).dag()] # target slot
                swap_ops += [nonnull_proj for _ in range(target + 1, source)] # slots in between
                swap_ops.append(qtp.basis(mode_dim, l) * qtp.basis(mode_dim, k).dag()) # source slot

                swap_block += qtp.tensor(swap_ops[::-1])

        ops.append(qtp.tensor(swap_block))

    ops += [null_proj for _ in range(source + 1, max_num_particles)]
                
    return qtp.tensor(ops[::-1])

def symmetrizer(ipart):
    full_op = qtp.Qobj()
    
    for itarg in range(ipart + 1):
        full_op += swap(ipart, itarg)
        
    return full_op

def creation(p):
    full_op = qtp.Qobj()
    
    for ipart in range(max_num_particles):
        full_op += 1. / np.sqrt(ipart + 1) * symmetrizer(ipart) * particle_creation(ipart, p)
        
    return full_op

## States

In [4]:
vacuum = qtp.tensor([null_ket] * max_num_particles)

def fock_state(occupancy):
    num_particles = sum(occ for p, occ in occupancy)
    if num_particles > max_num_particles:
        raise RuntimeError('Too many particles')
        
    modes_list = sum(([p] * occ for p, occ in occupancy), [])
    permutations = list(multiset_permutations(modes_list))
    
    zero_pad = [null_ket] * (max_num_particles - num_particles)

    state = qtp.Qobj()
    
    for perm in permutations:
        state += qtp.tensor(zero_pad + [particle_ket(p) for p in perm])
        
    state /= np.sqrt(len(permutations))
        
    return state



In [17]:
all_fock_states = []
fock_space_proj = qtp.Qobj()
for num_particles in range(max_num_particles + 1):
    for perm in multiset_permutations('p' * num_particles + 'd' * (num_modes - 1)):
        occupancy = []
        p = pmin
        occ = 0
        for l in perm:
            if l == 'p':
                occ += 1
            else:
                occupancy.append((p, occ))
                p += 1
                occ = 0

        occupancy.append((p, occ))

        state = fock_state(occupancy)

        all_fock_states.append(state)
        fock_space_proj += state.proj()

## Operator representation

In [6]:
def state_repr(istate, ipart):
    plabel = (istate >> num_mode_bits * ipart) & (mode_dim - 1)

    if plabel == null_mode:
        return r'\emptyset'
    else:
        return '{' + str((plabel + mode_dim // 2) % mode_dim - mode_dim // 2) + '}'

def print_op(qobj):
    if not qobj.isoper:
        raise RuntimeError('Qobj is not an operator')
        
    spmat = qobj.data.tolil()
    
    expr = ''
    for irow, icol in zip(*spmat.nonzero()):
        elem = spmat[irow, icol]
        if elem.imag != 0.:
            if expr:
                expr += ' + '
            expr += r'({:.2f}{:+.2f}i)'.format(elem.real, elem.imag)
            
        else:
            if elem.real < 0.:
                expr += ' - '
            elif expr:
                expr += ' + '

        expr += r'{:.2f}'.format(abs(elem.real))
            
        row_state = tuple(state_repr(irow, i) for i in range(max_num_particles))
        col_state = tuple(state_repr(icol, i) for i in range(max_num_particles))

        expr += r' | {} \rangle \langle {} |'.format(r'\,'.join(row_state[::-1]), r'\,'.join(col_state[::-1]))
        
    display(Math(expr))
    
def print_state(qobj):
    if not (qobj.isket or qobj.isbra):
        raise RuntimeError('Qobj is not a state')
        
    spmat = qobj.data.tolil()
    if qobj.isket:
        indices = spmat.nonzero()[0]
        data = spmat.toarray()[:, 0]
    else:
        indices = spmat.nonzero()[1]
        data = spmat.toarray()[0]
    
    expr = ''
    for ielem in indices:
        elem = data[ielem]
        if elem.imag != 0.:
            if expr:
                expr += ' + '
            expr += r'({:.2f}{:+.2f}i)'.format(elem.real, elem.imag)
            
        else:
            if elem.real < 0.:
                expr += ' - '
            elif expr:
                expr += ' + '

        expr += r'{:.2f}'.format(abs(elem.real))
            
        state = tuple(state_repr(ielem, i) for i in range(max_num_particles))

        if qobj.isket:
            expr += r' | {} \rangle'.format(r'\,'.join(state[::-1]))
        else:
            expr += r' \langle {} |'.format(r'\,'.join(state[::-1]))
            
    display(Math(expr))    

In [7]:
print_op(particle_creation(0, 1))

<IPython.core.display.Math object>

In [8]:
print_op(symmetrizer(0))

<IPython.core.display.Math object>

In [9]:
print_op(creation(1))

<IPython.core.display.Math object>

In [10]:
print_state(fock_state([(0, 2), (1, 1)]))

<IPython.core.display.Math object>

In [11]:
print_state(creation(1) * creation(0) * creation(1) * vacuum)

<IPython.core.display.Math object>

In [12]:
fock_state([(0, 2)]).dag() * ((creation(0) ** 2) * vacuum)

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[1.41421356]]

In [13]:
Ap_dag = creation(1)
Ap = Ap_dag.dag()

com = qtp.commutator(Ap, Ap_dag)

In [14]:
print_op(com * fock_space_proj)

<IPython.core.display.Math object>

In [15]:
print_state(Ap * Ap_dag * vacuum)

<IPython.core.display.Math object>

In [24]:
for state in all_fock_states:
    if (com * state - state).data.nonzero()[0].shape[0] != 0:
        print_state(state)
        print('->')
        print_state(com * state)

<IPython.core.display.Math object>

->


<IPython.core.display.Math object>

<IPython.core.display.Math object>

->


...

->


<IPython.core.display.Math object>

<IPython.core.display.Math object>

->


<IPython.core.display.Math object>